In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('/mnt/d/data/geo-reviews-dataset-2023.tskv', sep='\t', header=None)

In [8]:
df.columns = ['address', 'name', 'rating', 'rubrics', 'text']

In [9]:
df['address'] = df['address'].str.replace('address=', '', regex=False)
df['name'] = df['name'].str.replace('name_ru=', '', regex=False)
df['rating'] = df['rating'].str.replace('rating=', '', regex=False)
df['rubrics'] = df['rubrics'].str.replace('rubrics=', '', regex=False)
df['text'] = df['text'].str.replace('text=', '', regex=False)

In [10]:
df.dropna(inplace = True)

In [11]:
df = df[df['rubrics'].str.contains('Гостиница')] 

In [12]:
df = df[['name','rating','text']]

In [13]:
df['rating'] = df['rating'].apply(lambda x : int(x[0]))

In [14]:
df.head()

,name,rating,text
16,Бирюзовая Катунь,5,Очень крутые экскурсии. Обязательно приедем ещ...
48,Баунти,5,"Понравилось все! Гостеприимство, чистота, спок..."
77,Парк Отель,5,"Моментальное бронирование, быстрая регистрация..."
105,Меридиан,5,Хорошей отель. Девушка на ресепшен очень добро...
106,Меридиан,5,Гостиница понравилась. Отличное сочетание цены...


чистка

In [15]:
texts = df['text']

In [16]:
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt') 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/adas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/adas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
stopwords = stopwords.words('russian')

In [18]:
texts = texts.apply(lambda x: re.sub("[^а-яА-Я]"," ",x))
texts = texts.str.lower()
texts = texts.apply(lambda x: " ".join([word for word in x.split() if not word in set(stopwords)]))

In [19]:
texts

16        очень крутые экскурсии обязательно приедем ещ ...
48        понравилось гостеприимство чистота спокойствие...
77        моментальное бронирование быстрая регистрация ...
105       хорошей отель девушка ресепшен очень доброжела...
106       гостиница понравилась отличное сочетание цены ...
                                ...                        
499956    отличный отель необходимое халат зубные принад...
499957    красиво вс новое хорошие номера минусы плохая ...
499958    отеле остановились дороге домой москвы элисту ...
499959    утрам горячей воды номерах основном вс отлично...
499960    отель хороший чисто комфортно брал завтраком в...
Name: text, Length: 43398, dtype: object

In [20]:
from pymorphy3 import MorphAnalyzer
m = MorphAnalyzer()
lemm_texts  = texts.apply(lambda x: ' '.join([m.parse(word)[0].normal_form for word in x.split()]))

In [21]:
lemm_texts

16        очень крутой экскурсия обязательно приехать ещ...
48        понравиться гостеприимство чистота спокойствие...
77        моментальный бронирование быстрый регистрация ...
105       хороший отель девушка ресепшен очень доброжела...
106       гостиница понравиться отличный сочетание цена ...
                                ...                        
499956    отличный отель необходимый халат зубной принад...
499957    красиво вс новый хороший номер минус плохой шу...
499958    отель остановиться дорога домой москва элиста ...
499959    утро горячий вода номер основное вс отлично за...
499960    отель хороший чисто комфортно брать завтрак вп...
Name: text, Length: 43398, dtype: object

In [22]:
df['cl_txt'] = texts
df['lemm_txt'] = lemm_texts

In [23]:
df['tokens'] = df['lemm_txt'].apply(lambda x: list(x.split()))

In [24]:
#Формирование n-грамм
def ngrams(text, n):
    return [text[i:i+n] for i in range(len(text)-n+1)]

# формируем n-граммы в датаФрейме
df['unigrams']=df['tokens'].apply(lambda x: ngrams(x, 1))
df['bigrams']=df['tokens'].apply(lambda x: ngrams(x, 2))
df['trigrams']=df['tokens'].apply(lambda x: ngrams(x, 3))

In [25]:
df['main_parts'] = df['tokens'].apply(
    lambda tokens: [word for word in tokens if m.parse(word)[0].tag.POS in {'NOUN', 'ADJF', 'INFN'}]
)

## Рассмотрим 3 метода:

### TfidfVectorizer
    Оценивает важность слов с учетом их частоты в документе и редкости в корпусе.
### CountVectorizer
    Преобразует текст в векторы, где значения — это количество вхождений слов в документ. 
### HashingVectorizer
    Преобразует текст в фиксированную длину вектор, применяя хеширование для каждого слова

Выбираю TfidfVectorizer, так как он является наиболее эффективным и точным за счет своего алгоритма работы

In [26]:
df

,name,rating,text,cl_txt,lemm_txt,tokens,unigrams,bigrams,trigrams,main_parts
16,Бирюзовая Катунь,5,Очень крутые экскурсии. Обязательно приедем ещ...,очень крутые экскурсии обязательно приедем ещ ...,очень крутой экскурсия обязательно приехать ещ...,"[очень, крутой, экскурсия, обязательно, приеха...","[[очень], [крутой], [экскурсия], [обязательно]...","[[очень, крутой], [крутой, экскурсия], [экскур...","[[очень, крутой, экскурсия], [крутой, экскурси...","[крутой, экскурсия, приехать, посмотреть]"
48,Баунти,5,"Понравилось все! Гостеприимство, чистота, спок...",понравилось гостеприимство чистота спокойствие...,понравиться гостеприимство чистота спокойствие...,"[понравиться, гостеприимство, чистота, спокойс...","[[понравиться], [гостеприимство], [чистота], [...","[[понравиться, гостеприимство], [гостеприимств...","[[понравиться, гостеприимство, чистота], [гост...","[понравиться, гостеприимство, чистота, спокойс..."
77,Парк Отель,5,"Моментальное бронирование, быстрая регистрация...",моментальное бронирование быстрая регистрация ...,моментальный бронирование быстрый регистрация ...,"[моментальный, бронирование, быстрый, регистра...","[[моментальный], [бронирование], [быстрый], [р...","[[моментальный, бронирование], [бронирование, ...","[[моментальный, бронирование, быстрый], [брони...","[моментальный, бронирование, быстрый, регистра..."
105,Меридиан,5,Хорошей отель. Девушка на ресепшен очень добро...,хорошей отель девушка ресепшен очень доброжела...,хороший отель девушка ресепшен очень доброжела...,"[хороший, отель, девушка, ресепшен, очень, доб...","[[хороший], [отель], [девушка], [ресепшен], [о...","[[хороший, отель], [отель, девушка], [девушка,...","[[хороший, отель, девушка], [отель, девушка, р...","[хороший, отель, девушка, ресепшен, доброжелат..."
106,Меридиан,5,Гостиница понравилась. Отличное сочетание цены...,гостиница понравилась отличное сочетание цены ...,гостиница понравиться отличный сочетание цена ...,"[гостиница, понравиться, отличный, сочетание, ...","[[гостиница], [понравиться], [отличный], [соче...","[[гостиница, понравиться], [понравиться, отлич...","[[гостиница, понравиться, отличный], [понравит...","[гостиница, понравиться, отличный, сочетание, ..."
...,...,...,...,...,...,...,...,...,...,...
499956,AZIMUT Сити Отель Тула,5,"Отличный отель, есть все необходимое. Халат, з...",отличный отель необходимое халат зубные принад...,отличный отель необходимый халат зубной принад...,"[отличный, отель, необходимый, халат, зубной, ...","[[отличный], [отель], [необходимый], [халат], ...","[[отличный, отель], [отель, необходимый], [нео...","[[отличный, отель, необходимый], [отель, необх...","[отличный, отель, необходимый, халат, зубной, ..."
499957,AZIMUT Сити Отель Тула,4,"Красиво, всё новое, хорошие номера. Минусы: пл...",красиво вс новое хорошие номера минусы плохая ...,красиво вс новый хороший номер минус плохой шу...,"[красиво, вс, новый, хороший, номер, минус, пл...","[[красиво], [вс], [новый], [хороший], [номер],...","[[красиво, вс], [вс, новый], [новый, хороший],...","[[красиво, вс, новый], [вс, новый, хороший], [...","[вс, новый, хороший, номер, плохой, шумоизоляц..."
499958,AZIMUT Сити Отель Тула,5,В отеле остановились по дороге домой из Москвы...,отеле остановились дороге домой москвы элисту ...,отель остановиться дорога домой москва элиста ...,"[отель, остановиться, дорога, домой, москва, э...","[[отель], [остановиться], [дорога], [домой], [...","[[отель, остановиться], [остановиться, дорога]...","[[отель, остановиться, дорога], [остановиться,...","[отель, остановиться, дорога, москва, элиста, ..."
499959,AZIMUT Сити Отель Тула,4,4 только из-за того что по утрам нет горячей в...,утрам горячей воды номерах основном вс отлично...,утро горячий вода номер основное вс отлично за...,"[утро, горячий, вода, номер, основное, вс, отл...","[[утро], [горячий], [вода], [номер], [основное...","[[утро, горячий], [горячий, вода], [вода, номе...","[[утро, горячий, вода], [горячий, вода, номер]...","[утро, горячий, вода, номер, ос

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
vectorizer = TfidfVectorizer()
text_vectors = vectorizer.fit_transform(df['lemm_txt'])

Так как у нас есть только текстовые данные запросов и отзывов, без информации о пользователях, косинусное сходство идеально подходит для этой задачи. Оно позволяет эффективно измерять схожесть между текстами на основе их содержания, игнорируя различия в формулировках. Этот метод работает независимо от истории запросов и предпочтений пользователей, что делает его простым и эффективным инструментом для подбора категорий по текстовому содержанию.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
#построим модель
def model(request):
    request_vector = vectorizer.transform([request])
    similarities = cosine_similarity(request_vector, text_vectors)
    recommended_indexes = similarities.argsort()[0][-3:][::-1]  # Топ-3 рекомендаций
    rec = [df.iloc[idx]['name'] for idx in recommended_indexes]
    return rec

In [52]:
# проверим ее работу
r = model('Сочи гостинциа')
r   

['Tigre de Cristal Hotel & Resort', 'Сочи Парк Отель', 'Grand Gallery']

Метрика персонализации лучше других, потому что она оценивает, насколько хорошо система адаптирует рекомендации под индивидуальные предпочтения пользователей. Она предотвращает однообразие предложений и фокусируется на разнообразии, улучшая опыт пользователя и повышая вовлеченность. Это позволяет создать более уникальные и релевантные рекомендации для каждого пользователя.

In [60]:
def personalization_metric(recommendations):
    unique_recs = len(set([rec for recs in recommendations.values() for rec in recs])) # Уникальные рекомендации
    total_recs = len([rec for recs in recommendations.values() for rec in recs])  # Всего рекомендаций
    return unique_recs / total_recs  # Доля уникальных рекомендаций

# Пример использования
user_requests = df['text'].sample(n=50, random_state=42).tolist()
recommendations = {req: model(req) for req in user_requests}
print("Персонализация:", personalization_metric(recommendations))

Персонализация: 0.9266666666666666


In [62]:
import pickle

# Сохранение векторов и векторизатора в файл
with open('/mnt/d/data/text_vectors.pkl', 'wb') as f:
    pickle.dump(text_vectors, f)
    
# Сохранение векторизатора для дальнейшего использования
with open('/mnt/d/data/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

df[['lemm_txt','name','rating']].to_csv('/mnt/d/data/df.csv',index = False)
